## De-optimize cas13d

***

In [72]:
import pandas as pd
from joblib import load
from optimalcodon.seqoptimization.optimization import optimization
from optimalcodon.projects.rnastability.predictuncertainty import predict_seq_with_uncertainty
from optimalcodon.seqoptimization.mutate import translate

In [4]:
modelo = load("../../data/19-08-08-PredictiveModelStability/predictivemodel.joblib")

In [5]:
# cas13d sequence
seq = "ATGGcgagcgaggccagcatcgaaaaaaaaaagtccttcgccaagggcatgggcgtgaagtccacactcgtgtccggctccaaagtgtacatgacaaccttcgccgaaggcagcgacgccaggctggaaaagatcgtggagggcgacagcatcaggagcgtgaatgagggcgaggccttcagcgctgaaatggccgataaaaacgccggctataagatcggcaacgccaaattcagccatcctaagggctacgccgtggtggctaacaaccctctgtatacaggacccgtccagcaggatatgctcggcctgaaggaaactctggaaaagaggtacttcggcgagagcgctgatggcaatgacaatatttgtatccaggtgatccataacatcctggacattgaaaaaatcctcgccgaatacattaccaacgccgcctacgccgtcaacaatatctccggcctggataaggacattattggattcggcaagttctccacagtgtatacctacgacgaattcaaagaccccgagcaccatagggccgctttcaacaataacgataagctcatcaacgccatcaaggcccagtatgacgagttcgacaacttcctcgataaccccagactcggctatttcggccaggcctttttcagcaaggagggcagaaattacatcatcaattacggcaacgaatgctatgacattctggccctcctgagcggactgaggcactgggtggtccataacaacgaagaagagtccaggatctccaggacctggctctacaacctcgataagaacctcgacaacgaatacatctccaccctcaactacctctacgacaggatcaccaatgagctgaccaactccttctccaagaactccgccgccaacgtgaactatattgccgaaactctgggaatcaaccctgccgaattcgccgaacaatatttcagattcagcattatgaaagagcagaaaaacctcggattcaatatcaccaagctcagggaagtgatgctggacaggaaggatatgtccgagatcaggaaaaatcataaggtgttcgactccatcaggaccaaggtctacaccatgatggactttgtgatttataggtattacatcgaagaggatgccaaggtggctgccgccaataagtccctccccgataatgagaagtccctgagcgagaaggatatctttgtgattaacctgaggggctccttcaacgacgaccagaaggatgccctctactacgatgaagctaatagaatttggagaaagctcgaaaatatcatgcacaacatcaaggaatttaggggaaacaagacaagagagtataagaagaaggacgcccctagactgcccagaatcctgcccgctggccgtgatgtttccgccttcagcaaactcatgtatgccctgaccatgttcctggatggcaaggagatcaacgacctcctgaccaccctgattaataaattcgataacatccagagcttcctgaaggtgatgcctctcatcggagtcaacgctaagttcgtggaggaatacgcctttttcaaagactccgccaagatcgccgatgagctgaggctgatcaagtccttcgctagaatgggagaacctattgccgatgccaggagggccatgtatatcgacgccatccgtattttaggaaccaacctgtcctatgatgagctcaaggccctcgccgacaccttttccctggacgagaacggaaacaagctcaagaaaggcaagcacggcatgagaaatttcattattaataacgtgatcagcaataaaaggttccactacctgatcagatacggtgatcctgcccacctccatgagatcgccaaaaacgaggccgtggtgaagttcgtgctcggcaggatcgctgacatccagaaaaaacagggccagaacggcaagaaccagatcgacaggtactacgaaacttgtatcggaaaggataagggcaagagcgtgagcgaaaaggtggacgctctcacaaagatcatcaccggaatgaactacgaccaattcgacaagaaaaggagcgtcattgaggacaccggcagggaaaacgccgagagggagaagtttaaaaagatcatcagcctgtacctcaccgtgatctaccacatcctcaagaatattgtcaatatcaacgccaggtacgtcatcggattccattgcgtcgagcgtgatgctcaactgtacaaggagaaaggctacgacatcaatctcaagaaactggaagagaagggattcagctccgtcaccaagctctgcgctggcattgatgaaactgcccccgataagagaaaggacgtggaaaaggagatggctgaaagagccaaggagagcattgacagcctcgagagcgccaaccccaagctgtatgccaattacatcaaatacagcgacgagaagaaagccgaggagttcaccaggcagattaacagggagaaggccaaaaccgccctgaacgcctacctgaggaacaccaagtggaatgtgatcatcagggaggacctcctgagaattgacaacaagacatgtaccctgttcagaaacaaggccgtccacctggaagtggccaggtatgtccacgcctatatcaacgacattgccgaggtcaattcctacttccaactgtaccattacatcatgcagagaattatcatgaatgagaggtacgagaaaagcagcggaaaggtgtccgagtacttcgacgctgtgaatgacgagaagaagtacaacgataggctcctgaaactgctgtgtgtgcctttcggctactgtatccccaggtttaagaacctgagcatcgaggccctgttcgataggaacgaggccgccaagttcgacaaggagaaaaagaaggtgtccggcaattccggatccggagcggccgcttacccatacgatgttccagattacgcttaa"
seq = seq.lower()

I mutate the following sequences:

1. All the possible mutations
2. mutations in the las half (restriction sites excluded)

In [10]:
# define the regions to mutate
# I cannot mutate the restriction sites
region1 = list(range(seq.find('ctcgccgacac') // 3, seq.find('gagagcattga') // 3))
region2 = list(range(seq.find('ccaagctgtat') // 3, seq.find('ggtgtccg') // 3))
wholerange = list(range(0, len(seq) // 3))

invalid1 = [] # every position can be mutated

invalid2 = [x for x in wholerange if x not in region1 + region2]

In [11]:
# the model parameters
mdl_params = {'specie': 'fish', 'cell_type': 'embryo mzt',
              'datatype': 'aamanitin polya'}


In [8]:
optimization(seq, [], optimization=False, model=modelo, outdir='results_data/', outname='deoptimizationCas13d_allregion', **mdl_params)

runing iteration 1 of 30
selection from 2001 sequences
runing iteration 2 of 30
selection from 2001 sequences
runing iteration 3 of 30
selection from 2001 sequences
runing iteration 4 of 30
selection from 2001 sequences
runing iteration 5 of 30
selection from 2001 sequences
runing iteration 6 of 30
selection from 2001 sequences
runing iteration 7 of 30
selection from 2001 sequences
runing iteration 8 of 30
selection from 2001 sequences
runing iteration 9 of 30
selection from 2001 sequences
runing iteration 10 of 30
selection from 2001 sequences
runing iteration 11 of 30
selection from 2001 sequences
runing iteration 12 of 30
selection from 2001 sequences
runing iteration 13 of 30
selection from 2001 sequences
runing iteration 14 of 30
selection from 2000 sequences
runing iteration 15 of 30
selection from 2001 sequences
runing iteration 16 of 30
selection from 2001 sequences
runing iteration 17 of 30
selection from 2001 sequences
runing iteration 18 of 30
selection from 2001 sequences
r

'ATGGCTAGCGAGGCCAGCATCGAAAAAAAAAAGTCCTTCGCCAAGGGCATGGGCGTGAAGTCCACCCTCGTGTCCGGCTCCAAGGTGTACATGACAACATTCGCCGAAGGCAGCGACGCCAGGCTGGAAAAAATCGTGGAGGGCGACAGCATCAGAAGCGTGAATGAGGGCGAGGCCTTCAGCGCTGAAATGGCCGATAAAAACGCCGGCTATAAGATCGGCAACGCCAAATTCAGCCATCCTAAGGGGTACGCCGTGGTGGCTAACAACCCCCTGTATACAGGACCTGTCCAGCAGGATATGCTCGGCCTGAAGGAAACTCTGGAGAAGAGGTACTTCGGCGAAAGCGCCGATGGCAATGACAATATTTGTATCCAGGTGATCCACAACATCCTGGACATTGAAAAAATCCTCGCCGAATACATTACCAATGCCGCCTACGCAGTAAATAATATCTCCGGCCTGGATAAGGACATTATTGGATTCGGCAAGTTCTCCACAGTGTATACCTACGACGAATTCAAAGACCCCGAGCACCATAGGGCCGCATTCAACAATAACGATAAGCTCATCAACGCCATCAAGGCCCAGTATGACGAGTTCGACAACTTCCTCGATAACCCCAGACTCGGCTATTTCGGCCAGGCCTTTTTCAGCAAGGAGGGCAGAAATTACATTATCAATTACGGCAACGAATGTTATGACATACTGGCCCTCCTGAGCGGACTGCGGCACTGGGTGGTCCACAACAACGAGGAAGAGTCCAGGATCTCCAGGACCTGGCTCTACAACCTCGATAAGAACCTCGACAATGAATACATCTCCACTCTCAACTACCTCTACGACAGGATTACCAATGAGCTGACCAACTCCTTCAGCAAGAACTCCGCCGCCAACGTGAACTATATTGCCGAGACTCTGGGAATCAACCCTGCCGAATTTGCCGAACAATATTTCAGATTCAGCATTATGAAAGAACAAAAAAACCTCGGCTTCAAT

In [12]:
optimization(seq, invalid2, optimization=False, model=modelo, outdir='results_data/', outname='deoptimizationCas13d_2ndhalf', **mdl_params)

runing iteration 1 of 30
selection from 2001 sequences
runing iteration 2 of 30
selection from 2001 sequences
runing iteration 3 of 30
selection from 2001 sequences
runing iteration 4 of 30
selection from 2001 sequences
runing iteration 5 of 30
selection from 2001 sequences
runing iteration 6 of 30
selection from 2001 sequences
runing iteration 7 of 30
selection from 2001 sequences
runing iteration 8 of 30
selection from 2001 sequences
runing iteration 9 of 30
selection from 2001 sequences
runing iteration 10 of 30
selection from 2001 sequences
runing iteration 11 of 30
selection from 2001 sequences
runing iteration 12 of 30
selection from 2001 sequences
runing iteration 13 of 30
selection from 2001 sequences
runing iteration 14 of 30
selection from 2001 sequences
runing iteration 15 of 30
selection from 2001 sequences
runing iteration 16 of 30
selection from 2001 sequences
runing iteration 17 of 30
selection from 2001 sequences
runing iteration 18 of 30
selection from 2001 sequences
r

'ATGGCGAGCGAGGCCAGCATCGAAAAAAAAAAGTCCTTCGCCAAGGGCATGGGCGTGAAGTCCACACTCGTGTCCGGCTCCAAAGTGTACATGACAACCTTCGCCGAAGGCAGCGACGCCAGGCTGGAAAAGATCGTGGAGGGCGACAGCATCAGGAGCGTGAATGAGGGCGAGGCCTTCAGCGCTGAAATGGCCGATAAAAACGCCGGCTATAAGATCGGCAACGCCAAATTCAGCCATCCTAAGGGCTACGCCGTGGTGGCTAACAACCCTCTGTATACAGGACCCGTCCAGCAGGATATGCTCGGCCTGAAGGAAACTCTGGAAAAGAGGTACTTCGGCGAGAGCGCTGATGGCAATGACAATATTTGTATCCAGGTGATCCATAACATCCTGGACATTGAAAAAATCCTCGCCGAATACATTACCAACGCCGCCTACGCCGTCAACAATATCTCCGGCCTGGATAAGGACATTATTGGATTCGGCAAGTTCTCCACAGTGTATACCTACGACGAATTCAAAGACCCCGAGCACCATAGGGCCGCTTTCAACAATAACGATAAGCTCATCAACGCCATCAAGGCCCAGTATGACGAGTTCGACAACTTCCTCGATAACCCCAGACTCGGCTATTTCGGCCAGGCCTTTTTCAGCAAGGAGGGCAGAAATTACATCATCAATTACGGCAACGAATGCTATGACATTCTGGCCCTCCTGAGCGGACTGAGGCACTGGGTGGTCCATAACAACGAAGAAGAGTCCAGGATCTCCAGGACCTGGCTCTACAACCTCGATAAGAACCTCGACAACGAATACATCTCCACCCTCAACTACCTCTACGACAGGATCACCAATGAGCTGACCAACTCCTTCTCCAAGAACTCCGCCGCCAACGTGAACTATATTGCCGAAACTCTGGGAATCAACCCTGCCGAATTCGCCGAACAATATTTCAGATTCAGCATTATGAAAGAGCAGAAAAACCTCGGATTCAAT

In [14]:
best_2nd = 'ATGGCGAGCGAGGCCAGCATCGAAAAAAAAAAGTCCTTCGCCAAGGGCATGGGCGTGAAGTCCACACTCGTGTCCGGCTCCAAAGTGTACATGACAACCTTCGCCGAAGGCAGCGACGCCAGGCTGGAAAAGATCGTGGAGGGCGACAGCATCAGGAGCGTGAATGAGGGCGAGGCCTTCAGCGCTGAAATGGCCGATAAAAACGCCGGCTATAAGATCGGCAACGCCAAATTCAGCCATCCTAAGGGCTACGCCGTGGTGGCTAACAACCCTCTGTATACAGGACCCGTCCAGCAGGATATGCTCGGCCTGAAGGAAACTCTGGAAAAGAGGTACTTCGGCGAGAGCGCTGATGGCAATGACAATATTTGTATCCAGGTGATCCATAACATCCTGGACATTGAAAAAATCCTCGCCGAATACATTACCAACGCCGCCTACGCCGTCAACAATATCTCCGGCCTGGATAAGGACATTATTGGATTCGGCAAGTTCTCCACAGTGTATACCTACGACGAATTCAAAGACCCCGAGCACCATAGGGCCGCTTTCAACAATAACGATAAGCTCATCAACGCCATCAAGGCCCAGTATGACGAGTTCGACAACTTCCTCGATAACCCCAGACTCGGCTATTTCGGCCAGGCCTTTTTCAGCAAGGAGGGCAGAAATTACATCATCAATTACGGCAACGAATGCTATGACATTCTGGCCCTCCTGAGCGGACTGAGGCACTGGGTGGTCCATAACAACGAAGAAGAGTCCAGGATCTCCAGGACCTGGCTCTACAACCTCGATAAGAACCTCGACAACGAATACATCTCCACCCTCAACTACCTCTACGACAGGATCACCAATGAGCTGACCAACTCCTTCTCCAAGAACTCCGCCGCCAACGTGAACTATATTGCCGAAACTCTGGGAATCAACCCTGCCGAATTCGCCGAACAATATTTCAGATTCAGCATTATGAAAGAGCAGAAAAACCTCGGATTCAATATCACCAAGCTCAGGGAAGTGATGCTGGACAGGAAGGATATGTCCGAGATCAGGAAAAATCATAAGGTGTTCGACTCCATCAGGACCAAGGTCTACACCATGATGGACTTTGTGATTTATAGGTATTACATCGAAGAGGATGCCAAGGTGGCTGCCGCCAATAAGTCCCTCCCCGATAATGAGAAGTCCCTGAGCGAGAAGGATATCTTTGTGATTAACCTGAGGGGCTCCTTCAACGACGACCAGAAGGATGCCCTCTACTACGATGAAGCTAATAGAATTTGGAGAAAGCTCGAAAATATCATGCACAACATCAAGGAATTTAGGGGAAACAAGACAAGAGAGTATAAGAAGAAGGACGCCCCTAGACTGCCCAGAATCCTGCCCGCTGGCCGTGATGTTTCCGCCTTCAGCAAACTCATGTATGCCCTGACCATGTTCCTGGATGGCAAGGAGATCAACGACCTCCTGACCACCCTGATTAATAAATTCGATAACATCCAGAGCTTCCTGAAGGTGATGCCTCTCATCGGAGTCAACGCTAAGTTCGTGGAGGAATACGCCTTTTTCAAAGACTCCGCCAAGATCGCCGATGAGCTGAGGCTGATCAAGTCCTTCGCTAGAATGGGAGAACCTATTGCCGATGCCAGGAGGGCCATGTATATCGACGCCATCCGTATTTTAGGAACCAACCTGTCCTATGATGAGCTCAAGGCCCTCGCCGACACCTTTTCCCTGGACGAGAACGGAAACAAGCTCAAGAAAGGCAAGCACGGCATGCGGAATTTCATTATTAATAATGTGATCAGCAATAAAAGGTTCCACTACCTGATCAGATACGGCGACCCGGCTCACCTGCACGAGATCGCCAAAAACGAGGCCGTGGTGAAGTTCGTGCTCGGCAGGATCGCAGACATACAGAAAAAACAGGGCCAGAACGGCAAGAACCAGATTGACAGGTACTACGAAACCTGTATCGGCAAGGATAAGGGCAAGAGCGTGTCAGAAAAGGTGGACGCCCTCACTAAGATTATCACTGGAATGAACTACGACCAGTTCGACAAGAAAAGGAGCGTGATTGAGGACACCGGCAGGGAAAATGCCGAGAGGGAGAAGTTTAAAAAGATCATCAGCCTGTACCTCACCGTGATCTACCACATCCTCAAGAATATTGTTAATATCAATGCCAGGTACGTTATCGGCTTCCATTGTGTGGAGAGAGATGCCCAACTGTACAAGGAGAAAGGCTACGACATCAATCTCAAGAAACTGGAAGAGAAGGGATTCAGCTCAGTTACCAAGCTCTGTGCTGGCATCGATGAAACGGCCCCCGATAAGCGAAAGGACGTGGAAAAGGAGATGGCCGAACGAGCCAAGGAGAGCATTGACAGCCTCGAGAGCGCCAACCCCAAGCTGTATGCCAATTACATCAAATACAGCGACGAGAAGAAAGCCGAGGAGTTCACCAGGCAGATTAATAGGGAGAAGGCCAAAACCGCCCTGAACGCCTACCTGAGGAATACCAAGTGGAATGTGATCATCAGAGAGGACCTCCTGAGAATTGACAACAAGACATGTACCCTCTTCAGGAACAAGGCCGTCCACCTGGAAGTGGCCAGGTATGTTCACGCCTACATCAACGACATTGCCGAGGTCAATAGCTACTTTCAACTGTACCATTACATCATGCAGAGAATTATCATGAATGAGAGGTACGAGAAAAGCTCAGGAAAGGTGTCCGAGTACTTCGACGCTGTGAATGACGAGAAGAAGTACAACGATAGGCTCCTGAAACTGCTGTGTGTGCCTTTCGGCTACTGTATCCCCAGGTTTAAGAACCTGAGCATCGAGGCCCTGTTCGATAGGAACGAGGCCGCCAAGTTCGACAAGGAGAAAAAGAAGGTGTCCGGCAATTCCGGATCCGGAGCGGCCGCTTACCCATACGATGTTCCAGATTACGCTTAA'

In [21]:
best_2nd.find('ctcgag'.upper())

2391

In [109]:
insertion = 'ctcgagagcgccaaccccaagctgtatgccaattacatcaaatacagcgacgagaagaaagccgaggagttcaccaggcagattaacagggagaaggccaaaaccgccctgaacgcctacctgaggaacaccaagtggaatgtgatcatcagggaggacctcctgagaattgacaacaagacatgtaccctgttcagaaacaaggccgtccacctggaagtggccaggtatgtccacgcctatatcaacgacattgccgaggtcaattcctacttccaactgtaccattacatcatgcagagaattatcatgaatgagaggtacgagaaaagcagcggaaaggtgtccgagtacttcgacgctgtgaatgacgagaagaagtacaacgataggctcctgaaactgctgtgtgtgcctttcggctactgtatccccaggtttaagaacctgagcatcgaggccctgttcgataggaacgaggccgccaagttcgacaaggagaaaaagaaggtgtccggcaattccggatcc'
optimal30a = best_2nd[:2391] + insertion  + seq[2922:]

In [113]:
assert translate(optimal30a.upper()) == translate(best_2nd)

In [116]:
optimal30_b = best_2nd[:1704] + \
    'gagctcaaggccctcgccgacaccttttccctggacgagaacggaaacaagctcaagaaaggcaagcacggcatgagaaatttcattattaataacgtgatcagcaataaaaggttccactacctgatcagatacggtgatcctgcccacctccatgagatcgccaaaaacgaggccgtggtgaagttcgtgctcggcaggatcgctgacatccagaaaaaacagggccagaacggcaagaaccagatcgacaggtactacgaaacttgtatcggaaaggataagggcaagagcgtgagcgaaaaggtggacgctctcacaaagatcatcaccggaatgaactacgaccaattcgacaagaaaaggagcgtcattgaggacaccggcagggaaaacgccgagagggagaagtttaaaaagatcatcagcctgtacctcaccgtgatctaccacatcctcaagaatattgtcaatatcaacgccaggtacgtcatcggattccattgcgtcgagcgtgatgctcaactgtacaaggagaaaggctacgacatcaatctcaagaaactggaagagaagggattcagctccgtcaccaagctctgcgctggcattgatgaaactgcccccgataagagaaaggacgtggaaaaggagatggctgaaagagccaaggagagcattgacagcctcgag' +\
    best_2nd[2397:]

In [117]:
translate(seq.upper()) == translate(best_2nd)

True

In [118]:
translate(best_2nd) == translate(optimal30_b.upper())

True

In [130]:
data = {
    'seqs': [seq, best_2nd, optimal30_b, optimal30a],
    'name': ['original', '50%', '30% b', '30% a']
}
data = pd.DataFrame(data).assign(seqs = lambda x: x.seqs.str.upper())

In [131]:
# predict the stability
preds = data.seqs.apply(predict_seq_with_uncertainty, models=modelo, **mdl_params)

data['prediction'] = preds.map(lambda x: x[1])
data['ci_l'] = preds.map(lambda x: x[0])
data['ci_u'] = preds.map(lambda x: x[-1])

In [132]:
data.to_csv('results_data/cas13dS.csv', index=False)